# Tests for trace_operators

In [1]:
import random
from pto.core.base import Op, Dist, tracer


In [2]:

def random_program():
    return([tracer.sample('pos 1', Dist(random.random)),
            tracer.sample('pos 2', Dist(random.choice, ['a','b','c'])),
            tracer.sample('pos 3', Dist(random.randint, 1, 10))])

def fitness(sol): return sol[0]

In [3]:
sol = random_program()
fit = fitness(sol)
print(sol, fit)

[0.2024606295925796, 'b', 3] 0.2024606295925796


In [4]:
# test __init__ 

op = Op(random_program, fitness)
#op.bind(random_program, fitness)
repr(op)

"Op('random_program', 'fitness', 'mutate_point_ind', 'crossover_uniform_ind')"

In [5]:
# test create_ind

sol = op.create_ind()
sol

Sol(pheno=[0.04155542836401005, 'a', 3], geno={'pos 1': Dist(random, (),  val=0.04155542836401005), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=a), 'pos 3': Dist(randint, (1, 10),  val=3)})

In [6]:
# test evaluate_ind

fit = op.evaluate_ind(sol)
fit

0.04155542836401005

In [7]:
# test fix_ind

print('original:', sol.geno)
sol.geno['pos 1'] = Dist(random.randint, 1, 3, val = 2) # unfeasible change incompatible with generator
print('changed:', sol.geno)
op.fix_ind(sol.geno)
print('fixed:', sol.geno) 

original: {'pos 1': Dist(random, (),  val=0.04155542836401005), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=a), 'pos 3': Dist(randint, (1, 10),  val=3)}
changed: {'pos 1': Dist(randint, (1, 3),  val=2), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=a), 'pos 3': Dist(randint, (1, 10),  val=3)}
repair:  ('randint', (1, 3)) ('random', ())
fixed: {'pos 1': Dist(random, (),  val=0.08642946885246428), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=a), 'pos 3': Dist(randint, (1, 10),  val=3)}


In [8]:
# test mutate_position_wise_ind
print('parent before mutation:', sol.geno, '\n')
before = str(sol.geno)
off = op.mutate_position_wise_ind(sol)
print('parent after mutation:', sol.geno, '\n') 
after = str(sol.geno)
assert before == after
print('offspring:', off.geno, '\n') # expected one change on average

parent before mutation: {'pos 1': Dist(random, (),  val=0.08642946885246428), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=a), 'pos 3': Dist(randint, (1, 10),  val=3)} 

parent after mutation: {'pos 1': Dist(random, (),  val=0.08642946885246428), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=a), 'pos 3': Dist(randint, (1, 10),  val=3)} 

offspring: {'pos 1': Dist(random, (),  val=0.08642946885246428), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=a), 'pos 3': Dist(randint, (1, 10),  val=3)} 



In [9]:
# test mutate_point_ind
print('parent before mutation:', sol.geno, '\n')
before = str(sol.geno)
off = op.mutate_point_ind(sol)
print('parent after mutation:', sol.geno, '\n') 
after = str(sol.geno)
assert before == after
print('offspring:', off.geno, '\n') # expected one change on average

parent before mutation: {'pos 1': Dist(random, (),  val=0.08642946885246428), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=a), 'pos 3': Dist(randint, (1, 10),  val=3)} 

{'pos 1': Dist(random, (),  val=0.3101352525664377), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=a), 'pos 3': Dist(randint, (1, 10),  val=3)}
parent after mutation: {'pos 1': Dist(random, (),  val=0.08642946885246428), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=a), 'pos 3': Dist(randint, (1, 10),  val=3)} 

offspring: {'pos 1': Dist(random, (),  val=0.3101352525664377), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=a), 'pos 3': Dist(randint, (1, 10),  val=3)} 



In [10]:
# test mutate_random_ind
print('parent before mutation:', sol.geno, '\n')
before = str(sol.geno)
off = off = op.mutate_random_ind(sol)
print('parent after mutation:', sol.geno, '\n') 
after = str(sol.geno)
assert before == after
print('offspring:', off.geno, '\n') # expected one change on average

parent before mutation: {'pos 1': Dist(random, (),  val=0.08642946885246428), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=a), 'pos 3': Dist(randint, (1, 10),  val=3)} 

parent after mutation: {'pos 1': Dist(random, (),  val=0.08642946885246428), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=a), 'pos 3': Dist(randint, (1, 10),  val=3)} 

offspring: {'pos 1': Dist(random, (),  val=0.5310208655935545), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=c), 'pos 3': Dist(randint, (1, 10),  val=1)} 



In [11]:
sol1 = op.create_ind()
sol2 = op.create_ind()

In [12]:
# test crossover_one_point_ind

print(sol1.pheno, '\n')
print(sol2.pheno, '\n')
off = op.crossover_one_point_ind(sol1, sol2)
print(op.fix_ind(sol1.geno).pheno, '\n') # assert: sol1 before == sol1 after
print(op.fix_ind(sol2.geno).pheno, '\n') # assert: sol2 before == sol2 after
print(off.pheno, '\n') # expected: initial segment of off == sol1, rest of off == sol2

[0.5426016436873111, 'a', 7] 

[0.6406796989486875, 'c', 5] 

[0.5426016436873111, 'a', 7] 

[0.6406796989486875, 'c', 5] 

[0.5426016436873111, 'a', 7] 



In [13]:
# test crossover_uniform_ind

print(sol1.pheno, '\n')
print(sol2.pheno, '\n')
off = op.crossover_uniform_ind(sol1, sol2)
print(op.fix_ind(sol1.geno).pheno, '\n') # assert: sol1 before == sol1 after
print(op.fix_ind(sol2.geno).pheno, '\n') # assert: sol2 before == sol2 after
print(off.pheno, '\n') # expected: mix of sol1 and sol2

[0.5426016436873111, 'a', 7] 

[0.6406796989486875, 'c', 5] 

[0.5426016436873111, 'a', 7] 

[0.6406796989486875, 'c', 5] 

[0.6406796989486875, 'c', 5] 

